In [1]:
import os
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, UpSampling2D, Input, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import warnings
from PIL import Image
from shutil import rmtree
warnings.filterwarnings('ignore')

2024-06-05 21:43:09.727954: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-05 21:43:11.650525: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
path_original = '256X'
path_blurred = 'Zgaussowane'

# Funkcja do ładowania i przetwarzania obrazów
def load_images(path, size=(256, 256)):
    images = []
    for filename in sorted(os.listdir(path)):
        img_path = os.path.join(path, filename)
        img = load_img(img_path, target_size=size, color_mode='rgb')
        img = img_to_array(img) / 255.0  # Normalizacja obrazów do zakresu [0, 1]
        images.append(img)
    return np.array(images)

In [3]:
if os.path.isdir('256X/.ipynb_checkpoints'):
    rmtree('256X/.ipynb_checkpoints')
if os.path.isdir('Zgaussowane/.ipynb_checkpoints'):
    rmtree('Zgaussowane/.ipynb_checkpoints')

In [4]:
original_images = load_images(path_original)
blurred_images = load_images(path_blurred)

print(f'Liczba załadowanych oryginalnych obrazów: {len(original_images)}')
print(f'Liczba załadowanych zblurowanych obrazów: {len(blurred_images)}')

Liczba załadowanych oryginalnych obrazów: 57
Liczba załadowanych zblurowanych obrazów: 57


In [5]:
def build_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    # model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    # odel.add(UpSampling2D((2, 2)))
    model.add(Conv2D(3, (3, 3), activation='sigmoid', padding='same'))
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

input_shape = (256, 256, 3)
model = build_model(input_shape)
model.summary()

2024-06-05 21:43:14.765075: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-05 21:43:15.286804: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 256, 256, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 256, 256, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 256, 256, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 256, 256, 64)   │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 256, 256, 3)    │         1,731 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 335,683 (1.28 MB)

 Trainable params: 335,683 (1.28 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
def build_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(3, (3, 3), activation='sigmoid', padding='same'))
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

input_shape = (256, 256, 3)
model = build_model(input_shape)
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ up_sampling2d (UpSampling2D)    │ (None, 512, 512, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 512, 512, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 256, 256, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 256, 256, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 256, 256, 64)   │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 256, 256, 3)    │         1,731 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 188,099 (734.76 KB)

 Trainable params: 188,099 (734.76 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from sklearn.model_selection import train_test_split

ilosc_epok = 28
# Podział danych na treningowe i walidacyjne
X_train, X_val, y_train, y_val = train_test_split(blurred_images, original_images, random_state=42)

# Trenowanie modelu
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=ilosc_epok, batch_size=8)

# Zapisywanie modelu
# model.save('deblur_cnn_model.h5')
keras.saving.save_model(model, f'deblur_cnn_model_redux{ilosc_epok}.keras')

Epoch 1/28
6/6 ━━━━━━━━━━━━━━━━━━━━ 52s 8s/step - loss: 0.0873 - val_loss: 0.0714
Epoch 2/28
6/6 ━━━━━━━━━━━━━━━━━━━━ 37s 6s/step - loss: 0.0734 - val_loss: 0.0711
Epoch 3/28
6/6 ━━━━━━━━━━━━━━━━━━━━ 34s 6s/step - loss: 0.0613 - val_loss: 0.0513
Epoch 4/28
6/6 ━━━━━━━━━━━━━━━━━━━━ 44s 6s/step - loss: 0.0454 - val_loss: 0.0279
Epoch 5/28
6/6 ━━━━━━━━━━━━━━━━━━━━ 43s 7s/step - loss: 0.0238 - val_loss: 0.0237
Epoch 6/28
1/6 ━━━━━━━━━━━━━━━━━━━━ 3:48 46s/step - loss: 0.0152

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
for i in range(3):
    restored_image = model.predict(np.expand_dims(X_val[i], axis=0))

    restored_image = restored_image[0] * 255
    restored_image = restored_image.astype('uint8')

    restored_image_pil = Image.fromarray(restored_image)
    restored_image_pil.save(f'Restored/{i}_{ilosc_epok}.jpg')